In [1]:
from kubernetesTool import executeKubectlCmd
from kubernetesTool import toCmd
from kubernetesTool import executeLocalCmd
from kubernetesTool import createConfig
from kubernetesTool import getPort
from kubernetesTool import executePeerCmd

aubonmoulin-79666dbdd9-kvf9b
austriafactor-7fbf86dcf-znm8x
cc-deployer-746f97765-dmjnw
hl-fabric-data-share-deployment-7f7547d845-gq9sg
invoice-chaincode-external-6d6dcf7d47-jjqww
invoice-net-chaincode-external-54cbcd776-nczr6
lyon-impots-77c667fb49-qvq27
megustaolive-f477b7b59-mgm6f
orderer-org1-scray-org-d8b6656f7-q8g95
peer200-549cb588fc-99zw9
peer201-5dbc445fc-4zv27
peer203-5668fc99d5-fbgxp
peer204-5d759557b7-5dhwd
peer205-76cb86657-256w6
store-f5996757c-prqv5
aubonmoulin
austriafactor
cc-deployer
hl-fabric-data-share-deployment
invoice-chaincode-external
invoice-net-chaincode-external
lyon-impots
megustaolive
orderer-org1-scray-org
peer200
peer201
peer203
peer204
peer205
store
aubonmoulin
austriafactor
hl-fabric-cc-external-invoice
hl-fabric-cc-external-invoice-net
hl-fabric-data-share-service
kubernetes
lyon-impots
megustaolive
orderer-org1-scray-org
peer100
peer101
peer102
peer200
peer201
peer203
peer204
peer205
peer48
peer49
peer50
peer55
store
store2
30081Error from server (No

Error from server (NotFound): services "olya0" not found
Error from server (NotFound): services "olya0" not found


In [2]:
#!pip install ipynb
from ipynb.fs.full.facturxWidget import *

## addPeer

In [3]:


def createConfig(peername='',peer_listen_port='',peer_gossip_port='',ca_country='DE',ca_province='Baden',ca_locality='Bretten'):
    try:  
        output  = subprocess.check_output(['/home/jovyan/work/usr/bin/kubectl', 'create','configmap','hl-fabric-peer-' + peername,
                                          '--from-literal=hostname=kubernetes.research.dev.seeburger.de',
                                          '--from-literal=org_name=' + peername,
                                          '--from-literal=data_share=hl-fabric-data-share-service:80',
                                          '--from-literal=ca_country='  + ca_country,
                                          '--from-literal=ca_province=' + ca_province,
                                          '--from-literal=ca_locality=' + ca_locality,
                                          '--from-literal=CORE_PEER_ADDRESS=kubernetes.research.dev.seeburger.de:' + peer_listen_port,
                                          '--from-literal=CORE_PEER_GOSSIP_EXTERNALENDPOINT=kubernetes.research.dev.seeburger.de:' + peer_gossip_port,
                                          '--from-literal=CORE_PEER_LOCALMSPID=' + peername + 'MSP'])
        return str(output)
    except Exception as e:
        return str(e)  




def addPeer(peername,ca_country='DE',ca_province='Baden',ca_locality='Bretten'):  
    if peername in peers.keys():
        return peers[peername]
    # Create configuration for new peer
    executeLocalCmd(['/bin/bash','-c','cd /home/jovyan/scray-ledger/containers/ && /home/jovyan/scray-ledger/containers/configure-deployment.sh -n ' + peername])
    # Start service
    executeKubectlCmd(['apply', '-f', '/home/jovyan/scray-ledger/containers/target/' + peername + '/k8s-peer-service.yaml'])
    # Create peer configuration
    res = createConfig(peername=peername,
                       peer_listen_port=getPort(peername=peername, name='peer-listen'),
                       peer_gossip_port=getPort(peername=peername, name='peer-gossip'),
                       ca_country=ca_country,ca_province=ca_province,ca_locality=ca_locality)
    # Start new peer
    executeKubectlCmd(['apply', '-f','/home/jovyan/scray-ledger/containers/target/' + peername + '/k8s-peer.yaml'])
    peer = kubernetespeer(name=peername)
    peers[peername] = peer
    return peer
    
# create, delete numbered peers
def createPeers(base_name,start=0, end=10):
    for x in range(start,end):
        peername = base_name + '-' + str(x)
        addPeer(peername)
         
def deletePeers(name):
    for x in range(0,10):
        peerneme = name + str(x)
        deletePeer(peerneme)    
  
def deleteAllPeers(config):
    for item in config['items']:
        try:
            if 'org_name' in item['data'].keys():
                deletePeer(item['data']['org_name'])
        except Exception as e:
            print(item['data'])
            

### config            
            
import subprocess
def deleteConfig(peername=''):
    try:  
        output  = subprocess.check_output(['/home/jovyan/work/usr/bin/kubectl', 'delete','configmap','hl-fabric-peer-' + peername])
        return str(output)
    except Exception as e:
        return str(e)  
        
    

  
##### execute

def executePeerCmd(pod_name,cmd, cli='scray-peer-cli'):
    try:  
        #cmd1 = "source /root/.bashrc && "
        cmd1 = ['/home/jovyan/work/usr/bin/kubectl', 'exec','-t', pod_name, '-c', cli,'--', '/bin/sh','-c',cmd]
        #cmd1 = ['/home/jovyan/work/usr/bin/kubectl', 'exec','-t', pod_name, '-c', cli,'--', '/bin/sh','-c'] + cmd
        print(toCmd(cmd1))
        output  = subprocess.check_output(cmd1)
        #output  = subprocess.check_output(['/home/jovyan/work/usr/bin/kubectl', 'exec', '--stdin' ,'--tty', pod_name, '-c', cli,'--', '/bin/sh','-c',cmd])
        return output.decode('ascii')
    except Exception as e:
        return str(e)

In [4]:
import kubernetesTool

class kubernetespeer:
    def __init__(self, name='',prefix='peer'):
        self.name = name
        self.pod_name = kubernetesTool.getPod(name)
        self.listen_port = kubernetesTool.getPort(peername=name, name = prefix + '-listen')
        self.gossip_port = kubernetesTool.getPort(peername=name, name = prefix + '-gossip')
        # ????????
        #print('kubernetespeer addBashEnvToPeer')
        #addBashEnvToPeer(self)
    def getCertificate(self):
        try:
            return str(subprocess.check_output(['cat', self.crt]))[2:-3]   
        except Exception as e:
            return str(e)      
        
class ordererpeer(kubernetespeer):
    def __init__(self, name='',prefix='peer'):
        super().__init__(name,prefix)
        self.host = 'orderer.example.com'
        self.orderer_intern = 7050
        
    # can change
    def get_ip(self, app='orderer-org1-scray-org'):
        return executeKubectlCmd(['get', 'pods','-l', 'app=' + app,'-o','jsonpath=\'{.items[*].status.podIP}\''])   
    

In [5]:
def toCmd(strlist):
    return ''.join((str(e) + ' ') for e in strlist)[:-1]

#!/home/jovyan/work/usr/bin/kubectl exec --stdin --tty $POD_NAME  -c scray-peer-cli -- /bin/sh /mnt/conf/peer_join.sh $ORDERER_IP  $ORDERER_HOSTNAME $CHANNEL_NAME         
def joinHyperledger(peer,orderer_ip,orderer_hostname,orderer_port,channel_name,shared_fs_host,ext_peer_ip='10.15.136.41'):          
    cmd = toCmd(['/mnt/conf/peer_join.sh', orderer_ip,orderer_hostname,orderer_port,channel_name,shared_fs_host,ext_peer_ip])
    print(cmd)
    return executePeerCmd(peer.pod_name, cmd)

def endorse_peer(peer=None, orderer_ip='',channel_name='',shared_fs_host='',ext_peer_ip='10.15.136.41'):
    print(peer.name)
    peername = peer.name
    peer_host_name = peername + '.kubernetes.research.dev.seeburger.de'
    cmd = toCmd(['/mnt/conf/orderer/scripts/inform_existing_nodes.sh',orderer_ip,channel_name,peer.name,shared_fs_host,ext_peer_ip,peer_host_name]) 
    #print(cmd)
    return executePeerCmd(orderer.pod_name,cmd,cli='scray-orderer-cli')

def add_channel(name):
    cmd1 = ['/mnt/conf/orderer/scripts/create_channel.sh', name, 'orderer.example.com', '7050']
    cmd = toCmd(cmd1)
    return executePeerCmd(orderer.pod_name, cmd1, cli='scray-orderer-cli')

## invoice functions

In [6]:
#kubectl exec --stdin --tty $PEER_POD -c scray-peer-cli -- /bin/sh /mnt/conf/peer/add-invoice.sh  $CHANNEL_NAME $INVOICE_ID

def get_certificate_string(peername,user='User1'):
    return 'x509::CN=' + user + '@' + peername + '.kubernetes.research.dev.seeburger.de,OU=client,L=Bretten,ST=Baden,C=DE::CN=ca.' + peername + '.kubernetes.research.dev.seeburger.de,O=' + peername + '.kubernetes.research.dev.seeburger.de,L=Bretten,ST=Baden,C=DE'

def chaincode_add_invoice(peer,channel_name,invoice_id,buyer):
    strlist = ['/mnt/conf/peer/add-invoice.sh', channel_name,invoice_id,get_certificate_string(buyer)]
    print(toCmd(strlist))
    return executePeerCmd(peer.pod_name,toCmd(strlist)) 


In [7]:
import json

def getHostname(peer,user):
    splits = user.split('@')
    if len(splits) > 1:
        return splits[1]
    return executePeerCmd(peer.pod_name,'echo $HOSTNAME')[:-1]

def getExport_MSPCONFIGPATH(hostname,user='User1'):
    splits = user.split('@')
    if len(splits) > 1:
        user = splits[0]
    path = '/mnt/conf/organizations/peerOrganizations/' + hostname + '/users/' + user + '@' + hostname + '/msp/'
    return 'export CORE_PEER_MSPCONFIGPATH=' + path
    
def getArgList(asset):
    if isinstance(asset, dict):
        asset = asset.values()
    return '[' + ''.join(('\"' + str(e) + '\",') for e in asset)[:-1] + ']' 

def invoke(peer,callopt,user='User1'):
    global _out
    _out = widgets.Output()
    with _out:
        try:
            export_cmd = getExport_MSPCONFIGPATH(getHostname(peer,user),user=user)
            cmd0 = str('\'' + callopt + "\'")
            #strlist = ['peer', 'chaincode', 'invoke', '-o', 'orderer.example.com:30081', '--tls', '--cafile', '/tmp/tlsca.example.com-cert.pem','-C', channel_name, '-n', 'basic', '-c', cmd0]
            #strlist = ['peer', 'chaincode', 'invoke', '--waitForEventTimeout','1s' ,'-o', 'orderer.example.com:30081', '--tls', '--cafile', '/tmp/tlsca.example.com-cert.pem','-C', channel_name, '-n', 'basic', '-c', cmd0]
            strlist = ['peer', 'chaincode', 'invoke', '-o', 'orderer.example.com:30081', '--waitForEvent',  '--tls', '--cafile', '/tmp/tlsca.example.com-cert.pem','-C', channel_name, '-n', 'basic', '-c', cmd0]

            cmd = toCmd(strlist)
            cmd = export_cmd + '  && ' + cmd
            print(cmd)  
            output = executePeerCmd(peer.pod_name, cmd)
            if output.endswith ('\n'):
                return output[:-1],_out
            return output,_out
        except Exception as e:
            print('Exception:', e)
            if output.endswith ('\n'):
                return output[:-1],_out
            return output,_out
        
def query(peer, callopt, user='User1'):
    global _out
    _out = widgets.Output()
    with _out:
        try:
            export_cmd = getExport_MSPCONFIGPATH(getHostname(peer,user),user=user)
            cmd0 = str('\'' + callopt + "\'")
            strlist = ['peer', 'chaincode', 'query', '-C', channel_name, '-n', 'basic', '-c', cmd0]
            cmd = toCmd(strlist)
            cmd = export_cmd + '  && ' + cmd
            print(cmd)
            output = executePeerCmd(peer.pod_name, cmd)
            if output.endswith ('\n'):
                output=output[:-1]
            return json.loads(output),_out
        except Exception as e:
            print('Exception:', e)
            return output,_out
    

    
def getEmptyInvoice(peer):
    callopt= '{"function":"GetEmptyInvoice","Args":[' + ']}'
    return query(peer,callopt)       
    
def createInvoice(peer,arglist='', user='User1'):
    callopt= '{"function":"CreateInvoice","Args":' + arglist + '}'
    return invoke(peer,callopt,user=user)    
    
def listInvoices(peer, user='User1'):
    callopt= '{"function":"ListInvoices","Args":[' + ']}'
    return query(peer,callopt,user=user)    
    
def listInvoice(peer, id='asset1', user='User1'):
    callopt= '{"function":"ListInvoice","Args":' + getArgList([id]) + '}'
    return query(peer,callopt,user=user)    
    
def transferInvoice(peer, id='asset1',name='', user='User1'):
    callopt= '{"function":"TransferInvoice","Args":' + getArgList([id,name]) + '}'
    return invoke(peer,callopt,user=user)      
    
    
    
def receivedInvoice(peer, id='asset1', user='User1'):
    callopt= '{"function":"ReceivedInvoice","Args":' + getArgList([id]) + '}'
    return invoke(peer,callopt,user=user)     
    
def receivedOrder(peer, id='asset1', user='User1'):
    callopt= '{"function":"ReceivedOrder","Args":' + getArgList([id]) + '}'
    return invoke(peer,callopt,user=user)   

def receivedPayment(peer, id='asset1', payer='', user='User1'):
    callopt= '{"function":"ReceivedPayment","Args":' + getArgList([id,payer]) + '}'
    return invoke(peer,callopt,user=user)   

def taxReceived(peer, id='asset1', user='User1'):
    callopt= '{"function":"TaxReceived","Args":' + getArgList([id]) + '}'
    return invoke(peer,callopt,user=user)    
    
    
    
def getAllKeys(peer):
    callopt= '{"function":"GetAllKeys","Args":[' + ']}'
    return query(peer,callopt)     

def assetExists(peer, id='asset1'):
    #callopt= '{"function":"AssetExists","Args":[' + '\"' + id + '\"' + ']}'
    callopt= '{"function":"AssetExists","Args":' + getArgList([id]) + '}'
    return query(peer,callopt)
        
def deleteAsset(peer, asset='asset1'):
    callopt= '{"function":"DeleteAsset","Args":[' + '\"' + asset + '\"' + ']}'
    return invoke(peer,callopt)
    


def readAssetTest(peer, asset='asset1'):
    callopt= '{"function":"ReadAssetTest","Args":[' + '\"' + asset + '\"' + ']}'
    return query(peer,callopt)
    
def updateAsset(peer,arglist=None):
    callopt= '{"function":"UpdateAsset","Args":' + getArgList(asset) + '}'
    return invoke(peer,callopt)


def getSubmittingClientIdentity(peer, user='User1'):
    callopt= '{"function":"GetSubmittingClientIdentity","Args":[' + ']}'
    return query(peer,callopt,user=user)

def getMSPID(peer):
    callopt= '{"function":"GetMSPID","Args":[' + ']}'
    return query(peer,callopt)   

  
def getRoleTransactions(peer):
    callopt= '{"function":"GetRoleTransactions","Args":[' + ']}'
    return query(peer,callopt)   


def appendRole(peer,user='',role='Buyer', taxInspector=''):
    name = getSubmittingClientIdentity(peer, user=user)[0]
    print(name)
    callopt= '{"function":"AppendRole","Args":' + getArgList([name,role,taxInspector]) + '}'
    return invoke(peer,callopt)

def getRoles(peer, user='User1'):
    #callopt= '{"function":"GetRoles","Args":[' + ']}'
    callopt= '{"function":"GetRoles","Args":[' + ']}'
    return query(peer,callopt,user=user)

def getAllRoles(peer):
    callopt= '{"function":"GetAllRoles","Args":[' + ']}'
    return query(peer,callopt)

### Enroll users

In [8]:
import subprocess
def downloadToLocal(host='10.15.136.41:30080',user='scray', pwd='scray',
                    local_path='./', local_name='' ,remote_path='upload'):  
    try:    
        output = subprocess.check_output(['curl', '--user',user + ':' + pwd, 
                                               'http://' + host + '/' + remote_path + '/' + local_name, '-o', local_path + '/' + local_name])
        #return str(callProcess)[3:-3]
        return output.decode('ascii')
    except Exception as e:
        return str(e)     
   

def uploadFromLocal(host='10.15.136.41:30080',user='scray', pwd='scray',
                    local_path='./', local_name='' ,remote_path='upload'):
    try:    
        output = subprocess.check_output(['curl', '--user',user + ':' + pwd, 
                                          '-T', local_path + '/' + local_name, 
                                          'http://' + host + '/' + remote_path + '/' + local_name])
        return output.decode('ascii')
    except Exception as e:
        return str(e)   
                    
#downloadToLocal(local_path='./', local_name='org3_update_in_envelope.pb' ,remote_path='upload')

def downloadToLocalCmdList(host='10.15.136.41:30080',user='scray', pwd='scray',
                    local_path='./', remote_name='' ,remote_path='/'):  
    return ['curl', '--user',user + ':' + pwd, 'http://' + host + '/' + remote_path + '/' + remote_name, '-o', local_path + '/' + remote_name]


In [9]:
# download walletcreator to peers
def uploadWalletToolToSharepoint():
    uploadFromLocal(local_path='/home/jovyan/scray-ledger/tools/wallet-creator/target', local_name='wallet-creator-0.0.1-SNAPSHOT-jar-with-dependencies.jar' ,remote_path='/')
    uploadFromLocal(local_path='/home/jovyan/scray-ledger/tools/wallet-creator', local_name='cert-creator.sh' ,remote_path='upload')

def installPeerWalletTool(peer):
    executePeerCmd(peer.pod_name,toCmd(downloadToLocalCmdList(local_path='/tmp', remote_name='wallet-creator-0.0.1-SNAPSHOT-jar-with-dependencies.jar'))) 
    #executePeerCmd(peer.pod_name,toCmd(downloadToLocalCmdList(local_path='/tmp', remote_name='cert-creator.sh'))) 
    executePeerCmd(peer.pod_name,'apk add openssl')
    executePeerCmd(peer.pod_name,'apk add openjdk8-jre')
    # chmod 755
    
def createPeerUser(peer,user, admin='admin',
    cert='/mnt/conf/organizations/peerOrganizations/kubernetes.research.dev.seeburger.de/ca/ca.kubernetes.research.dev.seeburger.de-cert.pem',
    key='/mnt/conf/organizations/peerOrganizations/kubernetes.research.dev.seeburger.de/ca/priv_sk'):
        cmd = ['/tmp/cert-creator.sh','--cacert',cert,'--cakey',key,'--new-user-crt',user,'--organizational-unit',admin,
               '--create-wallet','true','--wallet-creator-lib-path', '/tmp']
        return executePeerCmd(peer.pod_name,toCmd(cmd))
    
def installWalletTool():
    _out = widgets.Output()
    with _out:
        for key, peer in peers.items():
            installPeerWalletTool(peer)
            strlist = ['curl', 'https://raw.githubusercontent.com/scray/scray-ledger/develop/tools/wallet-creator/cert-creator.sh', '-o', '/tmp/cert-creator.sh']
            cmd = toCmd(strlist)
            print(cmd)
            output = executePeerCmd(peer.pod_name,  cmd)
            executePeerCmd(peer.pod_name,'chmod 755 /tmp/cert-creator.sh')
            strlist = ['curl', 'https://raw.githubusercontent.com/scray/scray-ledger/develop/tools/wallet-creator/create-user.sh', '-o', '/tmp/create-user.sh']
            cmd = toCmd(strlist)
            print(cmd)
            output = executePeerCmd(peer.pod_name,  cmd)
            executePeerCmd(peer.pod_name,'chmod 755 /tmp/create-user.sh')
    
def addUser(peer,user,org,org2):
    _out = widgets.Output()
    with _out:
        #strlist = ['/tmp/create-user.sh', user, peer.name + '.kubernetes.research.dev.seeburger.de']
        strlist = ['/tmp/create-user.sh', user,org,org2 ]
        cmd = toCmd(strlist)
        print(cmd)
        output = executePeerCmd(peer.pod_name, 'cd /tmp && ' + cmd)    
    return output,_out    

In [10]:
def getPeerUsers(peer,org):
    #strlist = ['/bin/ls',  '/mnt/conf/organizations/peerOrganizations/' + peer.name + '.kubernetes.research.dev.seeburger.de/users/']
    strlist = ['/bin/ls',  '/mnt/conf/organizations/peerOrganizations/' + org + '/users/']
    #addUser(peers['aubonmoulin'],'Tony.Dubois','aubonmoulin.fr','aubonmoulin.kubernetes.research.dev.seeburger.de')[1] 
    
    print(toCmd(strlist))
    users=[]
    lines=executePeerCmd(peer.pod_name,toCmd(strlist)).splitlines() 
    for line in lines:
        #users.append(line.split('@')[0])
        users.append(line)
    return users

def getPeerOrganizations(peer):
    strlist = ['/bin/ls',  '/mnt/conf/organizations/peerOrganizations/']
    return executePeerCmd(peer.pod_name,toCmd(strlist)).splitlines() 

def get_peer_of_user(user):
    for key,peer in peers.items():
        orgs=getPeerOrganizations(peer)
        for org in orgs:
            print(peer.pod_name, org)
            users = getPeerUsers(peer,org)
            for ouser in users:
                if user == ouser:
                    return peer

#### Rollen

In [11]:
def deleteAllRoles(peer):
    keys = getAllKeys(peer)[0]
    for key in keys:
        if key.startswith('roles'):
            deleteAsset(peer,asset=key)[1]   
 
#deleteAllRoles(peer)
#keys = getAllKeys(peer)[0]
#keys

In [12]:
def getUsedRoles(roles):
    _all=[]
    for element in roles:
        for role in element['roles']:
            if role not in _all:
                _all.append(role)
    return _all

#getUsedRoles(roles)

### Block

In [13]:
import subprocess
def downloadToLocal(host='10.15.136.41:30080',user='scray', pwd='scray',
                    local_path='./', local_name='' ,remote_path='upload'):  
    try:    
        output = subprocess.check_output(['curl', '--user',user + ':' + pwd, 
                                               'http://' + host + '/' + remote_path + '/' + local_name, '-o', local_path + '/' + local_name])
        #return str(callProcess)[3:-3]
        return output.decode('ascii')
    except Exception as e:
        return str(e)     
  
#deleteRemote(host='10.15.136.41:30080',user='scray', pwd='scray', local_name='invoicing-block-19.block.json', remote_path='blocks/invoicing')
def deleteRemote(host='10.15.136.41:30080',user='scray', pwd='scray',
                    local_name='' ,remote_path='upload'):  
    try:    
        output = subprocess.check_output(['curl', '--user',user + ':' + pwd, '-X', 'DELETE',
                                               'http://' + host + '/' + remote_path + '/' + local_name])
        #return str(callProcess)[3:-3]
        return output.decode('ascii')
    except Exception as e:
        return str(e)    


In [14]:
import base64

def get_height():
    global _out
    _out = widgets.Output()
    with _out:
        strlist = ['/mnt/conf/peer/block-operator.sh', 'info', '--channel', channel_name]
        cmd = toCmd(strlist)
        print(cmd)
        output = executePeerCmd(peers[peernames[0]].pod_name,  cmd)
        return json.loads(output)['height']

def uploadBlockToShare(id):
    strlist = ['/mnt/conf/peer/block-operator.sh fetch', '--channel', channel_name, '--block', str(id),'--publish', shared_fs_host]
    cmd = toCmd(strlist)
    output = executePeerCmd(peers[peernames[0]].pod_name,  cmd)
    
def getBlockFromShare(id):
    filename = 'invoicing-block-' + str(id) + '.block.json'
    downloadToLocal(host='10.15.136.41:30080',user='scray', pwd='scray', local_path='/tmp/',local_name=filename, remote_path='blocks/invoicing')
    output = subprocess.check_output(['cat', '/tmp/' + filename])
    return json.loads(output.decode('ascii'))           
    
def decode_block_invoice(block):
    _transaction = block['data']['data'][0]['payload']['data']['actions'][0]['payload']['action']['proposal_response_payload']['extension']['results']['ns_rwset'][1]['rwset']['writes'][0]
    if _transaction['value'] != None:
        return json.loads(base64.b64decode(_transaction['value']).decode("utf-8"))
    
def decode_block_transaction(block):
    _transaction = _block['data']['data'][0]['payload']['data']['actions'][0]['payload']['chaincode_proposal_payload']['input']['chaincode_spec']['input']['args']
    result = []
    for arg in _transaction:
        result.append(base64.b64decode(arg).decode("utf-8"))
    return result        

### create invoice

In [15]:
## required functions // different notebook ??????

import pandas as pd

def get_invoices():
    pd.set_option('display.max_rows', 500)
    return pd.read_parquet('./invoices_facturx.parquet')

invoices = get_invoices()

def getPathValue(m,path=''):
    if(len(path)<2):
        return m
    splits = path[1:].split("/", 1)
    value=m.get(splits[0])
    #print(splits,value)
    if not isinstance(value, type(None)):
        if len(splits) > 1:
            return getPathValue(value,path='/' + splits[1])
    return value    

def assign_invoice_to_asset(_invoice,asset):
    #print(asset)

    asset['hash'] = hash(repr(sorted(_invoice.items()))) 
    monetary = getPathValue(_invoice,'/supplyChainTradeTransaction/applicableHeaderTradeSettlement/specifiedTradeSettlementHeaderMonetarySummation/')
    asset['invoiceNumber'] = getPathValue(_invoice,'/exchangedDocument/id')['value']
    asset['netto'] = monetary['taxBasisTotalAmount'][0]['value']
    asset['tax'] = monetary['taxTotalAmount'][0]['value']
    
    try:
        asset['taxExemptionReason']  = getPathValue(_invoice,'/supplyChainTradeTransaction/applicableHeaderTradeSettlement/applicableTradeTax')[0]['exemptionReason']['value']
    except Exception as e:
        asset['taxExemptionReason']  = ''
 
    asset['countryBuyer'] = getPathValue(_invoice,'/supplyChainTradeTransaction/applicableHeaderTradeAgreement/buyerTradeParty/postalTradeAddress/countryID')['value']
    asset['countryOrigin']   = getPathValue(_invoice,'/supplyChainTradeTransaction/applicableHeaderTradeAgreement/sellerTradeParty/postalTradeAddress/countryID')['value']
    

In [16]:
import unicodedata
def strip_accents(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
                  if unicodedata.category(c) != 'Mn')

In [17]:
def get_cert_cn(cert):
    return cert.split('=')[1].split(',')[0]

def get_CN_of_invoice(invoice,party):
    _name=strip_accents(getPathValue(invoice,'/supplyChainTradeTransaction/applicableHeaderTradeAgreement/' + party + '/definedTradeContact/')[0]['personName']['value'].replace(' ','.'))
    _email=getPathValue(invoice,'/supplyChainTradeTransaction/applicableHeaderTradeAgreement/' + party + '/definedTradeContact/')[0]['emailURIUniversalCommunication']['uriid']['value']
    return _name + '@' + _email.split('@')[1]

def update_invoice(asset=None,invoice=None,id=None,seller='Tony.Dubois',buyer='Stephanie.Hoarau'):
    assign_invoice_to_asset(invoice,asset)
    asset['ID']='asset' + str(id)
    asset['owner'] = getSubmittingClientIdentity(get_peer_of_user(seller), user=seller)[0]
    asset['buyer'] = getSubmittingClientIdentity(get_peer_of_user(buyer), user=buyer)[0]
    asset['taxExemptionReason'] = strip_accents(asset['taxExemptionReason'])

def createInvoices(invoices):
    for i in range(0,20):
        try:
            invoice = invoices.iloc[i]
            seller = get_CN_of_invoice(invoice,'sellerTradeParty')
            buyer  = get_CN_of_invoice(invoice,'buyerTradeParty')
            print(i,seller,buyer,'asset'+str(i))
            asset = getEmptyInvoice(peer)[0]
            update_invoice(asset=asset,invoice=invoice,id=i,seller=seller,buyer=buyer)
            print(i,asset)
            createInvoice(get_peer_of_user(seller),getArgList(asset),user=seller)[0]
        except Exception as e:
            pass
        
def deleteAllInvoices(peer):
    keys = getAllKeys(peer)[0]
    for key in keys:
        if key.startswith('asset'):
            deleteAsset(peer,asset=key)[1]        

# Demo Functions

In [18]:
def get_cert_cn(cert):
    return cert.split(',')[0].split('=')[1]
    #return cert.split('=')[1].split('@')[0]

def get_person_names_with_role(role,roles):
    result = []
    for person in roles:
        if role in person['roles']:
            #result.append(get_cert_cn(person['name']))
            result.append(person['name'])
    return result        
        
def get_person_certs_with_role(role,roles):
    result = []
    for person in roles:
        if role in person['roles']:
            result.append(person['name'])
    return result                     

In [19]:
def disable_all_fields():
    for key,value in _widget.container.fields.items():
        value.widget.disabled=True
        
def enable_fields_for_role(rolename):
    disable_all_fields()
    if rolename == 'Buyer':
        _widget.container.fields['received'].widget.disabled=False
        _widget.container.fields['receivedOrder'].widget.disabled=False

    elif  rolename == 'Seller':   
        _widget.container.fields['sold'].widget.disabled=False
        _widget.container.fields['claimPaid'].widget.disabled=False
        _widget.container.fields['claimPaidBy'].widget.disabled=False
        _widget.container.fields['owner'].widget.disabled=False
        
    elif  rolename == 'Factor':   
        #_widget.container.fields['sold'].widget.disabled=False
        _widget.container.fields['claimPaid'].widget.disabled=False
        _widget.container.fields['claimPaidBy'].widget.disabled=False
        #_widget.container.fields['owner'].widget.disabled=False
                
    elif  rolename == 'TaxInspector':    
        _widget.container.fields['taxReceived'].widget.disabled=False       

In [20]:
def create_dropdown_owner(_widget):
    _certs=[_widget.container.fields['owner'].widget.value]
    _certs = _certs + get_person_certs_with_role('Factor',roles)
    return widgets.Dropdown(options=_certs, value = _certs[0], description="owner", style=style, layout=layout)

def replace_owner_field_by_dropdown(_widget):
    _owners = create_dropdown_owner(_widget)
    _widget.container.fields['owner'] = Field(widget=_owners,path=_widget.container.fields['owner'].path)
    _widget.redisplay()

### role buyer

In [21]:
def init_fields_buyer(user=None,peer=None):
    #peer=get_peer_of_user(user)

    def received_changed(b):
        if b['name'] == 'value':
            receivedInvoice(peer, id=id, user=user)[1]

    def receivedOrder_changed(b):
        if b['name'] == 'value':
            receivedOrder(peer, id=id, user=user)[1]    

    _widget.container.fields['received'].widget.observe(received_changed)
    _widget.container.fields['receivedOrder'].widget.observe(receivedOrder_changed)
    enable_fields_for_role('Buyer')

### role owner / seller

In [22]:
def init_fields_seller(user=None,peer=None):
    #peer=get_peer_of_user(user)
    
    def sold_changed(b):
        if b['name'] == 'value':
            name=_widget.container.fields['owner'].widget.value
            transferInvoice(peer, id=id,name=name, user=user)[1]

    def claimPaid_changed(b):
        if b['name'] == 'value':
            receivedPayment(peer, id=id, payer= _widget.container.fields['claimPaidBy'].widget.value, user=user)

    enable_fields_for_role('Seller')    
    _widget.container.fields['sold'].widget.observe(sold_changed)
    _widget.container.fields['claimPaid'].widget.observe(claimPaid_changed)   
    replace_owner_field_by_dropdown(_widget)

### role tax inspector

In [23]:
def init_fields_taxinspector(user=None,peer=None):
    #peer=get_peer_of_user(user)

    def taxReceived_changed(b):
        if b['name'] == 'value':
            taxReceived(peer, id=id, user=user)

    _widget.container.fields['taxReceived'].widget.observe(taxReceived_changed)
    enable_fields_for_role('TaxInspector')

### init

In [24]:
# init users and peers of invoice


def init_parties_from_asset(asset,roles):
    global id, owner, buyer, factor,peer_buyer,peer_owner,peer_factor,taxinspector,peer_taxinspector
    
    id=asset['ID']
    owner=get_cert_cn(asset['owner'])
    buyer=get_cert_cn(asset['buyer'])

    factor=get_person_names_with_role('Factor',roles)[0].split(',')[0].split('=')[1]  

    peer_buyer=get_peer_of_user(buyer)
    peer_owner=get_peer_of_user(owner)
    peer_factor=get_peer_of_user(factor) 

    taxinspector = get_cert_cn(getRoles(peer_owner, user=owner)[0]['taxInspector'])
    peer_taxinspector=get_peer_of_user(taxinspector)
 
def init_parties_from_widget(asset,roles):    
    owner=get_cert_cn(_widget.container.fields['owner'].widget.value)
    buyer=get_cert_cn(_widget.container.fields['buyer'].widget.value)

    factor=get_person_names_with_role('Factor',roles)[0].split(',')[0].split('=')[1]  
    id=_widget.container.fields['ID'].widget.value
    peer_buyer=get_peer_of_user(buyer)
    peer_owner=get_peer_of_user(owner)
    peer_factor=get_peer_of_user(factor) 

    taxinspector = get_cert_cn(getRoles(peer_owner, user=owner)[0]['taxInspector'])
    peer_taxinspector=get_peer_of_user(taxinspector)

    #peer_taxinspector=get_peer_of_user(taxinspector)
    #taxinspector=get_person_names_with_role('TaxInspector',roles)[0].split(',')[0].split('=')[1] 
    
    

# Adressen

In [25]:
def create_address_widget(_invoice):
    _widget1 =  widgets.HBox([widgets.VBox([facturx_widgets(_invoice,'/exchangedDocumentContext', hideUnused=True, hide=True).output ,
                              facturx_widgets(_invoice,'/exchangedDocument', hideUnused=True, hide=True).output]) ,
                              facturx_widgets(_invoice,'/supplyChainTradeTransaction/applicableHeaderTradeAgreement/sellerTradeParty', hideUnused=True, hide=True).output ,
                              facturx_widgets(_invoice,'/supplyChainTradeTransaction/applicableHeaderTradeAgreement/buyerTradeParty', hideUnused=True, hide=True).output])
    #display (_widget1)
    return _widget1




# pod list

In [26]:
def create_pod_widget(_invoice):
    pod_list = []
    for key,peer in peers.items():
        pod_list.append(facturx_widgets(peer.__dict__,'/', hideUnused=True, hide=True).output)
    _widget = widgets.VBox(pod_list)
    return _widget

# Demo

### Init kubernetes peers (2)

In [27]:
orderer = ordererpeer(name='orderer-org1-scray-org',prefix='orderer')
orderer_ip=orderer.get_ip()
orderer_hostname=orderer.host
orderer_port= orderer.listen_port

channel_name='invoicing'
shared_fs_host='10.14.128.38:30080'

In [28]:
def addPeerFromKubernetes(peername):
    peer= kubernetespeer(peername)
    peers[peername] = peer

peers={}    
addPeerFromKubernetes('aubonmoulin')
addPeerFromKubernetes('lyon-impots')
addPeerFromKubernetes('austriafactor')
addPeerFromKubernetes('megustaolive')
peernames = list(peers.keys())
#peernames

# Demo

### Init kubernetes peers (2)

In [29]:
orderer = ordererpeer(name='orderer-org1-scray-org',prefix='orderer')
orderer_ip=orderer.get_ip()
orderer_hostname=orderer.host
orderer_port= orderer.listen_port

channel_name='invoicing9'
shared_fs_host='10.14.128.38:30080'

In [30]:
def addPeerFromKubernetes(peername):
    peer= kubernetespeer(peername)
    peers[peername] = peer

peers={}    
addPeerFromKubernetes('aubonmoulin')
addPeerFromKubernetes('lyon-impots')
addPeerFromKubernetes('austriafactor')
addPeerFromKubernetes('megustaolive')
peernames = list(peers.keys())
#peernames

## people and roles

In [31]:
shop_owner          = 'Tony.Dubois@aubonmoulin.fr'
shop_owner_peer     = get_peer_of_user(shop_owner)
shop_owner_identity = getSubmittingClientIdentity(shop_owner_peer, user=shop_owner)[0]
shop_owner_roles    = getRoles(shop_owner_peer, user=shop_owner)[0]

roles=getAllRoles(shop_owner_peer)[0]
roleTransactions=getRoleTransactions(shop_owner_peer)[0]
usedRoles=getUsedRoles(roles)

/home/jovyan/work/usr/bin/kubectl exec -t aubonmoulin-79666dbdd9-kvf9b -c scray-peer-cli -- /bin/sh -c /bin/ls /mnt/conf/organizations/peerOrganizations/
aubonmoulin-79666dbdd9-kvf9b aubonmoulin.fr
/bin/ls /mnt/conf/organizations/peerOrganizations/aubonmoulin.fr/users/
/home/jovyan/work/usr/bin/kubectl exec -t aubonmoulin-79666dbdd9-kvf9b -c scray-peer-cli -- /bin/sh -c /bin/ls /mnt/conf/organizations/peerOrganizations/aubonmoulin.fr/users/


In [37]:
def create_role_widget(shop_owner_roles):   
    style = {'description_width': '150px'}
    layout = {'width': '400px'}
    _widget = facturx_widgets(shop_owner_roles,'/', hideUnused=True) 
    style = {'description_width': '250px'}
    layout = {'width': '500px'}
    return _widget



## list

In [38]:
assets = listInvoices(shop_owner_peer, user=shop_owner)[0]

#asset = listInvoice(peer_owner, id='asset3', user=owner)[0]
#asset = listInvoice(peer_buyer, id='asset3', user=buyer)[0]
#asset = listInvoice(peer_factor, id='asset3', user=factor)[0]
#asset = listInvoice(shop_owner_peer, id='asset14', user=shop_owner)[0]

In [39]:
asset = assets[0]['Record']

In [40]:

def calc_keys(assets):
    _keys = []
    for asset in assets:
        _keys.append(asset['Key'])
    return _keys    

In [41]:
_keys = calc_keys(assets)

In [42]:
init_parties_from_asset(asset,roles)

/home/jovyan/work/usr/bin/kubectl exec -t aubonmoulin-79666dbdd9-kvf9b -c scray-peer-cli -- /bin/sh -c /bin/ls /mnt/conf/organizations/peerOrganizations/
aubonmoulin-79666dbdd9-kvf9b aubonmoulin.fr
/bin/ls /mnt/conf/organizations/peerOrganizations/aubonmoulin.fr/users/
/home/jovyan/work/usr/bin/kubectl exec -t aubonmoulin-79666dbdd9-kvf9b -c scray-peer-cli -- /bin/sh -c /bin/ls /mnt/conf/organizations/peerOrganizations/aubonmoulin.fr/users/
aubonmoulin-79666dbdd9-kvf9b hotelsaintdenis.re
/bin/ls /mnt/conf/organizations/peerOrganizations/hotelsaintdenis.re/users/
/home/jovyan/work/usr/bin/kubectl exec -t aubonmoulin-79666dbdd9-kvf9b -c scray-peer-cli -- /bin/sh -c /bin/ls /mnt/conf/organizations/peerOrganizations/hotelsaintdenis.re/users/
/home/jovyan/work/usr/bin/kubectl exec -t aubonmoulin-79666dbdd9-kvf9b -c scray-peer-cli -- /bin/sh -c /bin/ls /mnt/conf/organizations/peerOrganizations/
aubonmoulin-79666dbdd9-kvf9b aubonmoulin.fr
/bin/ls /mnt/conf/organizations/peerOrganizations/aubo

### old

In [ ]:
import ipywidgets as widgets
_out = widgets.Output()
with _out:
    from ipynb.fs.full.invoiceDemoLibrary import *

In [ ]:
assets = listInvoices(shop_owner_peer, user=shop_owner)[0]
_keys = calc_keys(assets)

In [ ]:
def get_cert_cn(cert):
    return cert.split(',')[0].split('=')[1]
    #return cert.split('=')[1].split('@')[0]

def get_person_names_with_role(role,roles):
    result = []
    for person in roles:
        if role in person['roles']:
            #result.append(get_cert_cn(person['name']))
            result.append(person['name'])
    return result        
        
def get_person_certs_with_role(role,roles):
    result = []
    for person in roles:
        if role in person['roles']:
            result.append(person['name'])
    return result                     

In [ ]:
_org = _cn.split('@')[1]
for key,peer in _blockchain.peers.items():
    if _org in peer.organizations.keys():
        if _user in peer.organizations[_org].users.keys():
            peer.organizations[_org].users[_user]            

In [ ]:
def get_current_peer_with_role(rolename):    
    if rolename == 'Buyer':
        return peer_buyer
    elif  rolename == 'Seller':   
        return peer_owner
    elif  rolename == 'Factor':   
        return peer_factor
    elif  rolename == 'TaxInspector':    
        return peer_taxinspector    

    
def reload_clicked1(i):
    global buyer,peer_buyer
    global asset
    
    user = get_current_user_with_role(roleOptions.value)
    peer = get_current_peer_with_role(roleOptions.value)
    
    asset = listInvoice(peer, id, user=user)
    
    with _out:
        buyer=get_cert_cn(asset['buyer'])
        peer_buyer=get_peer_of_user(buyer)
        
    widget_load_asset(_widget,asset)
    role_selected('')  
 
def _block_btn_reload_clicked1(i):    
    height=get_height()
    _block_text.value = str(height)
    with _block_out:
        clear_output()  
        uploadBlockToShare(height-1)    
        _block = getBlockFromShare(height-1)
        _invoice = decode_block_invoice(_block)
        _transaction = decode_block_transaction(_block)
        _t = get_transaction()[0]
        clear_output()  
        facturx_widgets(_t,'/', hideUnused=True, hide=False).output
        
       
